In [1]:
#coding=utf-8
import numpy as np
import pandas as pd

## 1.原始数据

In [2]:
## 用户列表
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\jzqb_users.csv').decode('utf-8')
data_users = pd.read_csv(path, names=['店铺id','time','list_user'])
data_users['店铺id'] = data_users['店铺id'].astype('int')
data_users.head()

,店铺id,time,list_user
0,1427428057,2016-12,12008793.0|374148780.0|681652277.0|986942317.0...
1,1427428057,2017-01,90857981.0|208848176.0|374148780.0|405732538.0...
2,1427428057,2017-02,107428165.0|1247536771.0|1477529249.0|16135164...
3,1427428057,2017-03,539570707.0|681652277.0|730486572.0|731066425....
4,1427428057,2017-04,170226649.0|271244048.0|539570707.0|569260294....


In [31]:
## 店铺-地区
import MySQLdb
conn=MySQLdb.connect(host='192.168.1.119',user='mysqlsmda',passwd='123456',db='smda',port=3306,charset='utf8')
sql = 'SELECT * FROM smda.tb_shoplocation'
data_shoplocation = pd.read_sql(sql, conn)
dict_shopid_location = dict(zip(data_shoplocation['shopid'].values,data_shoplocation['locationid'].values))
## 写入本地
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\data_shoplocation.csv').decode('utf-8')
data_shoplocation.to_csv(path, index=False)

In [4]:
## 销量
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\jzqb_sales.csv').decode('utf-8')
data_sales = pd.read_csv(path)
data_sales['店铺id'] = data_sales['店铺id'].astype('int')
## datasales加上地区信息
def getLocation(x):
    if dict_shopid_location.has_key(str(x)) == False: return -1
    return dict_shopid_location[str(x)]
data_sales['location'] = data_sales['店铺id'].apply(getLocation)
data_sales = data_sales[data_sales['location']!=-1]
data_sales.head()

,店铺id,time,销量,location
0,1427428057,2016-12,15,8388264bc5092cca
1,1427428057,2017-01,30,8388264bc5092cca
2,1427428057,2017-02,8,8388264bc5092cca
3,1427428057,2017-03,27,8388264bc5092cca
4,1427428057,2017-04,28,8388264bc5092cca


## 2.数据预处理

In [5]:
# 删去低销量的店铺(平均销量处于后10%的，即平均销量<=7.0的删去)
temp = data_sales.groupby('店铺id').mean().sort_values('销量')
temp = temp[temp['销量']>7]
ids = temp.index
data_sales = data_sales[data_sales['店铺id'].isin(ids)]
data_sales.shape

(34614, 4)

In [6]:
# 删去数据缺失月份较多的店铺(共24个月，后10%的阈值是7个月)
temp = data_sales.groupby('店铺id').count().sort_values('time')
temp = temp[temp['time']>7]
ids = temp.index
data_sales = data_sales[data_sales['店铺id'].isin(ids)]
data_sales.shape

(33094, 4)

In [7]:
## 掐头去尾
data_sales = data_sales[(data_sales['time']!='2015-11')&(data_sales['time']!='2017-10')]
data_sales.shape

(30838, 4)

## 3.数据准备

In [8]:
# 每地区求每个月-企业数量
data = data_sales.groupby(['time','location']).count().reset_index()
data['key'] = data['location'] + data['time']
dict_locationmonth_companycount = dict(zip(data['key'].values, data['销量'].values))

In [9]:
# 求每地区每个月-市场规模
data = data_sales.groupby(['time','location']).sum().reset_index()
data['key'] = data['location'] + data['time']
dict_locationmonth_marketscale = dict(zip(data['key'].values, data['销量'].values))

In [10]:
# 每月企业所占本地区市场份额
data_sales['share'] = data_sales[['time','location','销量']]\
    .apply(lambda x: float(x[2])/dict_locationmonth_marketscale[x[1]+x[0]], axis=1)

In [11]:
# 每地区每个月-市场中平均每家企业的销量
dict_locationmonth_marketavgsale = {}
for key in dict_locationmonth_marketscale:
    dict_locationmonth_marketavgsale[key] = dict_locationmonth_marketscale[key] / dict_locationmonth_companycount[key]

In [12]:
# 每地区每个月-市场中平均每家企业的市场份额
dict_locationmonth_marketavgshare = {}
for key in dict_locationmonth_marketscale:
    dict_locationmonth_marketavgshare[key] = 1.0 / dict_locationmonth_companycount[key]

In [13]:
# 其它数据
## list_shop
list_shop = list(set(data_sales['店铺id'].values))
print len(list_shop)
## list_month
list_month = sorted(list(set(data_sales['time'].values)))

2408


In [14]:
# 给sales和users数据加索引，加快选择速度
data_sales = data_sales.set_index(['店铺id','time'])
data_users = data_users.set_index(['店铺id','time'])

## 4.识别竞争对手

In [15]:
N = 3 #窗口期
def getUsers(shopid, range_time):
    result = ''
    month_count = 0
    for time_index in range_time:
        try: #会有索引不到的问题，这里简陋点直接try跳过错误
            result += data_users.ix[shopid, list_month[time_index]].values[0] + '|'
            month_count += 1
        except:
            pass
    result = set(result.split('|'))
    return result, len(result), month_count
def identify_competitor(time_index):
    range_time = range(time_index-N, time_index)
    ## 每个shop的用户列表
    dict_shop_user = {}
    dict_shop_usercount = {}
    dict_shop_monthcount = {}
    for shopid in list_shop:
        result = getUsers(shopid, range_time)
        dict_shop_user[shopid] = result[0]
        dict_shop_usercount[shopid] = result[1]
        dict_shop_monthcount[shopid] = result[2]
    ## 计算共现
    results = []
    for shop1 in list_shop:
        for shop2 in list_shop:
            if shop1 == shop2: continue
            if dict_shop_monthcount[shop1] < N: continue #窗口期内每个月都要有销量
            if dict_shop_monthcount[shop2] < N: continue #窗口期内每个月都要有销量
            if dict_shop_usercount[shop1] < 50: continue #窗口期内用户数量至少大于50
            if dict_shop_usercount[shop2] < 50: continue #窗口期内用户数量至少大于50
            if dict_shop_usercount[shop2] < dict_shop_usercount[shop1]*0.1: continue #竞企用户数至少超过10%
            coincide = len(dict_shop_user[shop1] & dict_shop_user[shop2])
            stress = float(coincide) / dict_shop_usercount[shop1]
            if stress < 0.1: continue #不记录stress小于阈值的了，不然读取太慢
            result = str(shop1) + ',' + str(shop2) + ',' + str(list_month[time_index]) + ',' + str(coincide) + ','\
                    + str(stress)
            results.append(result)
    return results

In [16]:
# # 识别竞争对手
# ## 循环，计算每个时间前N个月的竞争关系
# list_coincide = []
# for time_index in range(N, len(list_month)):
#     print time_index
#     list_coincide.extend(identify_competitor(time_index))
# ## 将coincide写入txt
# path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\coincide_new_' + str(N) + '.csv').decode('utf-8')
# f = open(path, 'w')
# f.write('shop,competitor,time,coincide,stress\r\n')
# for line in list_coincide:
#     f.write(line + '\r\n')
# f.close()

In [17]:
# 读取识别的竞争对手
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\coincide_new_' + str(N) + '.csv').decode('utf-8')
data_coincide = pd.read_csv(path)

In [18]:
# 用竞争数据更新下list_shop
## list_shop
list_shop = list(set(data_coincide['shop'].values))
print len(list_shop)

2090


## 5.实证(检验识别的是不是竞争对手)

In [19]:
competitor_section = [0.1,0.2,0.3,0.4,1] ##竞争强度区间
## 获得竞争对手绩效之和
def getCompetitorPerformanceSum(shopid, index_section, time, target):
    _sum = 0.0
    data = data_coincide[(data_coincide['shop']==shopid)&(data_coincide['time']==time)\
                        &(data_coincide['stress']>=competitor_section[index_section])\
                        &(data_coincide['stress']<competitor_section[index_section+1])]
    competitors = list(data['competitor'].values)
    for c in competitors:
        try: ##目前这里的报错是因为没有索引值
            _sum += data_sales.xs((c, time))[target]
        except:
            pass
    return _sum
## 获得竞争对手绩效之平均值
def getCompetitorPerformanceAvg(shopid, index_section, time, target):
    _sum = 0.0
    _count = 0
    data = data_coincide[(data_coincide['shop']==shopid)&(data_coincide['time']==time)\
                        &(data_coincide['stress']>=competitor_section[index_section])\
                        &(data_coincide['stress']<competitor_section[index_section+1])]
    competitors = list(data['competitor'].values)
    for c in competitors:
        try: ##目前这里的报错是因为没有索引值
            _sum += data_sales.xs((c, time))[target]
            _count += 1
        except:
            pass
    if _count == 0: return 0 ##避免除数为0的情况
    return _sum/_count
print 'sum_competitors_performance: ' + str(getCompetitorPerformanceSum(1486938201, 0, '2016-02','share'))
print 'avg_competitors_performance: ' + str(getCompetitorPerformanceAvg(1486938201, 0, '2016-02','share'))

sum_competitors_performance: 0.0
avg_competitors_performance: 0


In [20]:
## 获得本期绩效
def getNowPerformance(shopid, time, target):
    result = 0
    try: #避免没有索引的情况
        result = data_sales.xs((shopid, time))[target]
    except:
        pass
    return result
## 获得上期绩效
def getLastPerformance(shopid, time, target):
    result = 0
    try:
        result = data_sales.xs((shopid, list_month[list_month.index(time)-1]))[target]
    except:
        pass
    return result
print getNowPerformance(1427428057, '2017-02', 'share')
print getLastPerformance(1427428057, '2017-03', 'share')

0.0380952380952
0.0380952380952


In [21]:
## 获得企业前N个月平均销量，做控制变量
def getEnterpriseSale(shopid, time_index):
    times = range(time_index-N, time_index)
    _sum = 0.0
    _count = 0.0
    for time in times:
        try:
            _sum += data_sales.loc[shopid, list_month[time]]['销量']
            _count += 1
        except:
            pass
    if _count == 0: return 0
    return _sum/_count
## 获得企业前N个月平均市场规模，做控制变量
def getEnterpriseShare(shopid, time_index):
    times = range(time_index-N, time_index)
    _sum = 0.0
    _count = 0.0
    for time in times:
        try:
            _sum += data_sales.loc[shopid, list_month[time]]['share']
            _count += 1
        except:
            pass
    if _count == 0: return 0
    return _sum/_count
print getEnterpriseSale(1427428057,18)
print getEnterpriseShare(1427428057,18)

28.6666666667
0.119870617788


In [22]:
## 获得市场规模，做控制变量
def getMarketSaleSum(shopid,time):
    key = dict_shopid_location[str(shopid)] + time
    return dict_locationmonth_marketscale[key]
## 获得平均市场规模，做控制变量
def getMarketSaleAvg(shopid, time):
    key = dict_shopid_location[str(shopid)] + time
    return dict_locationmonth_marketavgsale[key]
def getMarketShareAvg(shopid, time):
    key = dict_shopid_location[str(shopid)] + time
    return dict_locationmonth_marketavgshare[key]
print getMarketSaleSum(1427428057,'2017-02')
print getMarketSaleAvg(1427428057,'2017-02')
print getMarketShareAvg(1427428057,'2017-02')

210
35
0.166666666667


In [23]:
# 输出 月份，shopid，自身本期绩效，自身上期绩效，0级竞争者市场份额加总，1级竞争者，2级竞争者..
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\evidence_new_03.csv').decode('utf-8')
f = open(path, 'w')
f.write('time,shopid,performance_c,enterprise_sale,enterprise_share_c,'\
        + 'marketsalesum,marketsaleavg,marketshareavg_c,'\
        + 'competitor0_c,competitor1_c,competitor2_c,competitor3_c\r\n')
for shopid in list_shop:
    for time_index in range(N,len(list_month)):
        time = list_month[time_index]
        time_last = list_month[time_index-1]
        performance_now = getNowPerformance(shopid, time,'share')
        performance_last = getLastPerformance(shopid, time,'share')
        if performance_now == 0: continue # 去除本月绩效为0的情况
        if performance_last == 0: continue # 去除上月绩效为0的情况
        list_competitor = [getCompetitorPerformanceSum(shopid, i, time, 'share') for i in range(4)]
        list_competitor_last = [getCompetitorPerformanceSum(shopid, i, time_last, 'share') for i in range(4)] #上期竞争对手绩效
        f.write(time + ',' + str(shopid) + ',' + str(performance_now-performance_last) + ','\
                + str(getEnterpriseSale(shopid,time_index)) + ','
                + str(getEnterpriseShare(shopid,time_index)-getEnterpriseShare(shopid,time_index-1)) + ','\
                + str(getMarketSaleSum(shopid,time)) + ',' + str(getMarketSaleAvg(shopid,time)) + ','\
                + str(getMarketShareAvg(shopid,time)-getMarketShareAvg(shopid,time_last)) + ','\
                + str(list_competitor[0]-list_competitor_last[0]) + ','\
                + str(list_competitor[1]-list_competitor_last[1]) + ','\
                + str(list_competitor[2]-list_competitor_last[2]) + ','\
                + str(list_competitor[3]-list_competitor_last[3]) + '\r\n')
print 'done'
f.close()

done


In [24]:
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\evidence_new_03.csv').decode('utf-8')
data = pd.read_csv(path)
data.head()

,time,shopid,performance_c,enterprise_sale,enterprise_share_c,marketsalesum,marketsaleavg,marketshareavg_c,competitor0_c,competitor1_c,competitor2_c,competitor3_c
0,2016-03,1459224577,0.002021,354.666667,0.003752,23533,146,-0.001365,0.0,0.0,0.0,0.0
1,2016-04,1459224577,-0.000080,321.333333,-0.012814,25445,138,-0.000776,0.0,0.0,0.0,0.0
2,2016-05,1459224577,-0.003751,302.666667,-0.006496,30641,158,-0.000253,0.0,0.0,0.0,0.0
3,2016-06,1459224577,-0.004931,391.000000,-0.000603,31455,154,-0.000255,0.0,0.0,0.0,0.0
4,2016-07,1459224577,-0.000434,342.000000,-0.002921,39364,187,-0.000164,0.0,0.0,0.0,0.0


In [25]:
data.sort_values('competitor0_c',ascending=False).head()

,time,shopid,performance_c,enterprise_sale,enterprise_share_c,marketsalesum,marketsaleavg,marketshareavg_c,competitor0_c,competitor1_c,competitor2_c,competitor3_c
11264,2016-11,1508872008,0.003634,52.333333,0.001592,2441,59,0.002168,0.949907,0.000000,0.000000,0.000000
5251,2016-08,1485080311,0.013976,29.666667,0.002356,2671,60,0.000000,0.895939,0.000000,0.000000,0.000000
1650,2016-06,1430856203,-0.001252,61.000000,0.000085,4989,113,0.000000,0.866547,0.064141,0.094809,0.000000
2709,2016-06,1550615457,-0.003264,32.666667,0.000761,4989,113,0.000000,0.847807,0.158950,0.026057,0.174638
11165,2017-07,1671073566,-0.001617,27.000000,0.001354,9896,77,0.000061,0.801397,0.035267,0.000000,0.000000


In [26]:
_mean = data['performance_c'].mean()
print _mean
data = data[data['performance_c']>_mean]

-0.000605558877112


In [27]:
data.shape

(13677, 12)

In [28]:
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\evidence_new_test.csv').decode('utf-8')
data.to_csv(path,index=False)